In [59]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [18]:
print(tf.__version__)

2.12.0


In [19]:
df = pd.read_csv('Churn_Modelling.csv')

In [20]:
X = df.iloc[:, 3:-1 ].values
y = df.iloc[:, -1 ].values

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# y= le.fit_transform(y)
X[:, 2] = le.fit_transform(X[:, 2])

In [26]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers= [('encoder', OneHotEncoder(), [1])], remainder= 'passthrough')
X = np.array(ct.fit_transform(X))
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [29]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [30]:
from keras.engine.sequential import Sequential
ann = Sequential()

##Adding First hidden layer

In [31]:
ann.add(tf.keras.layers.Dense(units=6, activation= 'relu'))

##Adding Second hidden layer

In [33]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

##Adding the Output Layer

In [34]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

##Training the ANN

In [35]:
ann.compile(optimizer='adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
ann.fit(X_train, y_train, batch_size= 32, epochs= 100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.6333 - accuracy: 0.7237
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5176 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4757 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4598 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4487 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4385 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4287 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4195 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4113 - accuracy: 0.7960
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4041 - accura

##Sample Prediction##
Country: France, </br>
Credit_Score: 600, </br>
Gender: Male, </br>
Age: 40, </br>
Tenure: 3, </br>
Account_Balance: 30000, </br>
No._of_Products: 2, </br>
No._of_Credit_Cards: 1, </br>
Is_Active_Member: Yes,  </br>
Estimated_Salary: 50000 

In [40]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) >0.5)

1/1 [==============================] - 0s 86ms/step
[[False]]


In [74]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
df_outcome = pd.DataFrame({'predicted': y_pred.flatten(), 'actual':y_test.flatten()})
df_outcome
# print(y_test.reshape(len(y_test),1))

63/63 [==============================] - 0s 2ms/step


,predicted,actual
0,False,0
1,False,1
2,False,0
3,False,0
4,False,0
...,...,...
1995,False,0
1996,False,0
1997,False,0
1998,False,0


In [75]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm, accuracy_score(y_pred, y_test)

(array([[1586,    9],
        [ 347,   58]]),
 0.822)